# Power Analysis with Clustered Standard Errors and Pre-Experiment Data

This script guides you through an advanced power analysis with clustered standard errors[1] and pre-experiment data[2].

Experimental power is overstated when it fails to account for clustered data which violates the assumption of independence within data. This method adjusts standard errors to account for clusters (in this case by User ID).

Pre-experiment data on the other hand can be exploited to improve experimental power be reducing variance in the dependent variable.

1. http://cameron.econ.ucdavis.edu/research/Cameron_Miller_JHR_2015_February.pdf

2. https://exp-platform.com/Documents/2013-02-CUPED-ImprovingSensitivityOfControlledExperiments.pdf

In [1]:
import math
from scipy import stats
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

### Parameters


In [38]:
mde = 0.10 # minimum detectable effect size (proportion)
power = 0.8
alpha = 0.05 # significance threshold

cluster = 'user_id'
dv = 'clicked' #dependent variable
cat_features = ['platform'] #enter categorical features here (to make dummies)

### Load Data

In [46]:
url = 'https://raw.githubusercontent.com/t-boeck/exp-thom/main/data/historical_clickthrough_data.csv'
df = pd.read_csv(url)
df.head()

,clicked,platform,user_rating,user_id
0,1,iOS,5,1001
1,0,iOS,3,1002
2,1,iOS,3,1003
3,0,iOS,1,1004
4,0,iOS,1,1005


In [47]:
df = sm.add_constant(df)
df = pd.get_dummies(df, columns=cat_features, drop_first=True)
X = df.drop(columns=[dv, cluster])

mod = sm.OLS(df[dv], X).fit()

df['resid'] = mod.resid

In [43]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                clicked   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.1940
Date:                Fri, 06 May 2022   Prob (F-statistic):              0.824
Time:                        18:09:40   Log-Likelihood:                -360.19
No. Observations:                 500   AIC:                             726.4
Df Residuals:                     497   BIC:                             739.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5460      0.051     10.636      0.000       0.445       0.647
user_rating     -0.0047      0.016     -0.294      0.769      -0.036       0.027
platform_iOS     0.0255      0.046      0.559      0.576      -0.064       0.115
==============================================================================
Omnibus:                     2133.148   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.110
Skew:                          -0.201   Prob(JB):                     8.97e-19
Kurtosis:                       1.043   Cond. No.                         8.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
clustered_mod = sm.OLS(df['resid'], df['const']).fit().get_robustcov_results(
                                        'cluster', groups=df[cluster],
                                        use_correction=True, df_correction=True)

#calculate standard deviation: standard error * square root of n
sd = clustered_mod.bse[0] * np.sqrt(df.shape[0])

#compute absolute effect from relative effect
absolute_effect_size = abs(df[dv].mean()
                           - df[dv].mean()*(1+mde))

#normalize effect size relative to standard deviation
effect_size = absolute_effect_size / sd

recommended_n = int(sm.stats.tt_ind_solve_power(effect_size=effect_size,
                                alpha=alpha, power=power, alternative='larger'))

print("Desired relative MDE is {}.".format(mde))
print("Desired absolute MDE is {}.".format(round(absolute_effect_size, 4)))
print("Required sample size in each group: {:,}.".format(recommended_n))

Desired relative MDE is 0.1.
Desired absolute MDE is 0.055.
Required sample size in each group: 1,273.
